# Extending built in classes

This tutorial covers how to extend built-in classes to further customize your use of the package to suit your own unique data processing/anaysis needs.

Let's say there is a particular analysis use-case that is not provided as a built-in in the current package release, by following the object-oriented structure of the package you can quickly create and add methods to extend the functionality of all existing class structures.